In [1]:
import numpy as np
from scipy import stats
import scipy.optimize as opt
import converge
import numerical

In [2]:
pdf = stats.multivariate_normal((23, 7), 4).pdf
fun = lambda x: -pdf(x)

In [3]:
opt.minimize(fun, [1, 1]).x

array([1, 1])

In [4]:
import scipy.optimize as opt
from converge import Converge
from numpy.linalg import inv

def minimize(f, theta, g, H, epsilon = 0.0000001, maxsteps = 100):
    converge = Converge(f, epsilon, maxsteps)
    
    while not converge.done(theta):
        d = -inv(H(theta)).dot(g(theta))
        theta += d * opt.minimize(lambda eta: f(theta + eta * d), 1).x
    return theta

In [5]:
minimize(fun, [1, 1], numerical.grad(fun), numerical.hess(fun)) 

array([-285193.07162766,  976720.95542707])